In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Reshape
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
keras.__version__

In [ ]:
%matplotlib inline

In [ ]:
IMAGENET_MEANS = [0.40760392, 0.45795686, 0.48501961]
# IMAGENET_MEANS = [103.939, 116.779, 123.68]

def process_image(img):
    processed_image = np.array(img).astype(np.float32)
    processed_image /= 255
    for x in range(3):
        processed_image[:, :, x] -= IMAGENET_MEANS[x]
    return processed_image
        
def restore_image(img):
    restored_image = np.array(img)
    for x in range(3):
        restored_image[:, :, x] += IMAGENET_MEANS[x]
    restored_image.clip(0, 1)
    restored_image *= 255
    return restored_image.astype(np.uint8)

In [ ]:
img = cv2.imread("../../data/images/Amsterdam.jpg", 1)
img = cv2.resize(img, (224, 224))

processed_img = process_image(img)

In [ ]:
# plt.figure(figsize=(10, 10))
plt.imshow(img[:, :, [2, 1, 0]])

In [ ]:
vgg16 = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
layer_pos_dict = {"conv1_2" : 2, "conv2_2" : 5, "conv3_2" : 8, "conv4_2" : 12, "conv5_2" : 16}
layer_pick = "conv3_2"

In [ ]:
get_target_layer = K.function([vgg16.layers[0].input],
                              [vgg16.layers[layer_pos_dict[layer_pick]].output])
img_input = np.expand_dims(processed_img, axis=0)
target_layer = get_target_layer([img_input])[0]

In [ ]:
input_proxy = Input(shape=(1,))
image_pixels = Dense(224 * 224 * 3, use_bias=False)(input_proxy)
input_image = Reshape((224, 224, 3))(image_pixels)
last_layer = input_image
for i in range(1, layer_pos_dict[layer_pick] + 1):
    next_layer = vgg16.layers[i](last_layer)
    last_layer = next_layer
    
structure_rebound_model = Model(inputs=input_proxy, outputs=last_layer)

In [ ]:
structure_rebound_model.summary()

In [ ]:
structure_rebound_model.compile(optimizer="adam", loss="mse")

In [ ]:
structure_rebound_model.fit(np.ones((1,)), [target_layer], epochs=100, verbose=1)

In [ ]:
recover_image = K.function([structure_rebound_model.layers[0].input],
                           [structure_rebound_model.layers[2].output])
output_image = recover_image([np.ones((1,1))])[0][0, :, :, :]
restored_image = restore_image(output_image)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
plt.imshow(restored_image[:, :, [2, 1, 0]])

plt.subplot(1, 2, 2)
plt.imshow(img[:, :, [2, 1, 0]])